# Phishing Email Generation and Classification

Experimental notebook for classification and generation of phishing emails using several techniques.

Roadmap:
- Preprocessing
    - Sentiment Analysis
- Classification
    - Logistic Regression on emails
    - word2vec model
    - Transformer-based model
- Generation
    - With GANs
    - With T5
    - With transformers
- Adversarial Learning
    - EA-based method of classifying
    - Cyclical GANs
    - Use EA to seed an NN?

In [3]:
# Install requirements - uncomment this line the first time you run this notebook
#!pip install -r ./requirements.txt

In [27]:
import pandas as pd
import numpy as np
from functools import reduce
from tqdm import tqdm
# from .autonotebook import tqdm as notebook_tqdm
import itertools
from collections import Counter

In [5]:
myseed = 10897

# Email-Spam-Dataset

Use this dataset as preliminary exploration of classification techniques. Classifier can then be applied to Enron dataset if it works

0: not spam
1: spam

In [6]:
spam_col_names = ['id', 'Body', 'Label']

In [7]:
dfSA = pd.read_csv('./kaggle-datasets/Email-Spam-Dataset/completeSpamAssassin.csv', names=spam_col_names)

In [8]:
dfEnron = pd.read_csv('./kaggle-datasets/Email-Spam-Dataset/enronSpamSubset.csv', names=spam_col_names)

In [9]:
dfLing = pd.read_csv('./kaggle-datasets/Email-Spam-Dataset/lingSpam.csv', names=spam_col_names)

In [10]:
#merge datasets
dfs = [dfSA, dfEnron, dfLing]
dfSpam = pd.concat(dfs).dropna(axis=0)

Take a peek at the body of the emails

In [11]:
dfSpam.head()

,id,Body,Label
1.0,0.0,\r\nSave up to 70% on Life Insurance.\r\nWhy S...,1
2.0,1.0,1) Fight The Risk of Cancer!\r\nhttp://www.adc...,1
3.0,2.0,1) Fight The Risk of Cancer!\r\nhttp://www.adc...,1
4.0,3.0,##############################################...,1
5.0,4.0,I thought you might like these:\r\n1) Slim Dow...,1


# Preprocessing

In avoidance of 'self-plagiarisation', much of the code for the `Dataset` class is adapted from my Text Processing Sentiment Analysis assignment. Code can be provided to markers on request

In [12]:
#nltk
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords as NLTK_STOP
import string

In [13]:
#download wordnet for lemmatization
#uncomment appropriate line if you get error: "Resource wordnet not found.", "Resource punkt not found.", etc...

# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')

In [14]:
#spacy imports
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS as SPACY_STOP

#download spacy dataset
#uncomment the line below if you get error "Can't find model 'en_core_web_sm'"
# ! python -m spacy download en_core_web_sm

nlp = spacy.load('en_core_web_sm')

In [15]:
class Dataset:
    def __init__(self, df, preprocessing=["lower"], feature_selection=["alltokens"], retList=False):
        
        #read preprocessing and feature_selection configuration
        self.preprocessing = preprocessing
        self.feature_selection = feature_selection
        self.data = df
        self.retList = retList

        # === define various processors and regexes for various preprocessing/feature selection methods ===

        # NLTK Stemming Engine
        self.porter = PorterStemmer()

        #NLTK Lemmatizing Engine
        self.wn_lt = WordNetLemmatizer()

        #tokenizers

        #words regex - splits on word boundaries, doesn't include punctuation etc
        self.word_tokenizer = RegexpTokenizer(r'\w+')

        #create tokenizer based on NLTK-provided regex from Labs
        nltk_pat = r'''(?x) # set flag to allow verbose regexps
            (?:[A-Z]\.)+ # abbreviations, e.g. U.S.A.
            | \w+(?:-\w+)* # words with optional internal hyphens
            | \$?\d+(?:\.\d+)?%? # currency and percentages, e.g. $12.40, 82%
            | \.\.\. # ellipsis
            | [][.,;"'?():_`-]
            | [>]?[:;][\']?[\(\)\[\]]+ # these are separate tokens; includes ], [
            '''
        self.nltk_tokenizer = RegexpTokenizer(nltk_pat)

        #create tokenizer based on custom regex based on above, with less features
        custom_pat = r'''(?x)
                \w+(?:-\w+)*
                |\$?\d+(?:\.\d+)?%?
                |\.\.\.'''
        self.custom_tokenizer = RegexpTokenizer(custom_pat)

        # stop words - SPACY_STOP defined above in imports

        self.NLTK_ENGLISH_STOP = set(NLTK_STOP.words('english'))

        # === apply preprocessing and feature selection ===

        self.process_phrases()
    
    def preprocess_phrase(self, phrase):
        """define preprocessing function for phrases
        can call any number of these options - however, some may not combine well"""

        if self.preprocessing == []:
            #no preprocessing
            return phrase
        if "lower" in self.preprocessing:
            #lowercase
            phrase = phrase.lower()
        if "newlines" in self.preprocessing:
            phrase = " ".join(filter(None, phrase.split("\n")))
        if "punc" in self.preprocessing:
            #remove punctuation
            phrase = phrase.translate(str.maketrans('','',string.punctuation))
        if "stemming" in self.preprocessing:
            #use NLTK stemming
            phrase = self.porter.stem(phrase)
        if "nltk_lemmatize" in self.preprocessing:
            #use NLTK's lemmatization method
            new_phrase = ""
            words = nltk.word_tokenize(phrase)
            for word in words:
                new_phrase += self.wn_lt.lemmatize(word)
            phrase = new_phrase
        if "spacy_lemmatize" in self.preprocessing:
            #use spacy's lemmatization method
            nlp_phrase = nlp(phrase)
            new_phrase = ""
            for token in nlp_phrase:
                new_phrase += (token.lemma_ + " ")
            phrase = new_phrase

        return phrase

    def extract_features_from_phrase(self, phrase):
        """define feature extraction function for phrases
        extracts all words from all phrases as features for document set
        for each method, if one has already been applied the phrase must be treated as a list"""

        #list of negation words from 
        negation_words = ["neither", "never", "no", "nobody", "none", "noone", "nor", "not", "nothing", "nowhere"]

        intensifer_words = ["absolutely", "completely", "extremely", "highly", "rather", "really", "so", "too", "totally", "utterly", "very"]

        #use if len(self.feature_selection) > 1 to check if any preprocessing already occurred, as input will be in a list

        if self.feature_selection == []:
            #we have to do some feature selection, absolute minimum is alltokens
            return phrase.split(" ")
        if "alltokens" in self.feature_selection:
            #split based on whitespace
            phrase = phrase.split(" ")
        if "nltk_tokenize" in self.feature_selection:
            #tokenize using NLTK tokenizer, with words regex
            if isinstance(phrase, list):
                phrase =  list(itertools.chain.from_iterable([self.word_tokenizer.tokenize(partial_phrase) for partial_phrase in phrase]))
            else:
                phrase = self.word_tokenizer.tokenize(phrase)
                # print(phrase)
        if "nltk_tokenize_2" in self.feature_selection:
            #tokenize using NLTK's regex
            if isinstance(phrase, list):
                phrase =  list(itertools.chain.from_iterable([self.nltk_tokenizer.tokenize(partial_phrase) for partial_phrase in phrase]))
            else:
                phrase = self.nltk_tokenizer.tokenize(phrase)
        if "custom_tokenize" in self.feature_selection:
            #tokenize with custom regex
            if isinstance(phrase, list):
                phrase = list(itertools.chain.from_iterable([self.custom_tokenizer.tokenize(partial_phrase) for partial_phrase in phrase]))
            else:
                phrase = self.custom_tokenizer.tokenize(phrase)
        if "nltk_stoplist" in self.feature_selection:
            #use an NLTK stoplist
            if isinstance(phrase, list):#check words if already split into list
                #TODO: quicker way to run this?
                phrases = []
                for partial_phrase in phrase:
                    if partial_phrase not in self.NLTK_ENGLISH_STOP:
                        phrases.append(partial_phrase)
                phrase = phrases
            else:#split into list then eval words
                phrase = [word for word in phrase.split(" ") if word not in self.NLTK_ENGLISH_STOP]
        if "spacy_stoplist" in self.feature_selection:
            #use a spacy stoplist
            if isinstance(phrase, list):
                phrases = []
                for partial_phrase in phrase:
                    phrases +=  [word for word in partial_phrase.split(" ") if word not in list(SPACY_STOP)]
                phrase = list(itertools.chain.from_iterable(phrases))
            else:
                phrase = [word for word in phrase.split(" ") if word not in list(SPACY_STOP)]
        if "nltk_pos_tag" in self.feature_selection:
            #use POS tagging - must be a list, so split by spaces if not already
            if isinstance(phrase, list):
                phrase = nltk.pos_tag(phrase)
            else:
                phrase = phrase.split(" ")
                phrase = nltk.pos_tag(phrase)
        if "negation_bigrams" in self.feature_selection:
            #create bigrams with negation words + their successors
            if len(self.feature_selection) > 1:
                phrases = []
                for j in range(0, len(phrase)):
                    partial_phrase = phrase[j]
                    split = partial_phrase.split(" ")

                    for i in range(0, len(split)):
                        if split[i] in negation_words and i != len(split) - 1:
                            #add the negation word and its successor to the list of phrases
                            phrases.append(split[i] + " " + split[i+1])
                        else:
                            #skip to next partial if negation is at end of partial phrase
                            if split[i] in negation_words and j != len(phrase) - 1:
                                # print("Next partial")
                                phrases.append(split[i] + " " + phrase[j+1].split(" ")[0])
                            else:
                                #skip entirely if this is the last partial phrase
                                phrases.append(split[i])
                phrase = phrases
            else:
                split = phrase.split(" ")
                phrase = []
                for i in range(0, len(split)):
                    if split[i] in negation_words and i != len(split) - 1:
                        phrase.append(split[i] + " " + split[i+1])
                    else:
                        phrase.append(split[i])
        if "intensifier_bigrams" in self.feature_selection:
            #create bigrams with intensifier words + their successors
            if len(self.feature_selection) > 1:
                phrases = []
                for j in range(0, len(phrase)):
                    partial_phrase = phrase[j]
                    split = partial_phrase.split(" ")

                    for i in range(0, len(split)):
                        if split[i] in intensifer_words and i != len(split) - 1:
                            #add the intensifier word and its successor to the list of phrases
                            phrases.append(split[i] + " " + split[i+1])
                        else:
                            #skip to next partial if intensifier is at end of partial phrase
                            if split[i] in intensifer_words and  j != len(phrase) - 1:
                                phrases.append(split[i] + phrase[j+1].split(" ")[0])
                            else:
                                #skip entirely if this is the last partial phrase
                                phrases.append(split[i])
                phrase = phrases
            else:
                split = phrase.split(" ")
                phrase = []
                for i in range(0, len(split)):
                    if split[i] in intensifer_words and i != len(split) - 1:
                        phrase.append(split[i] + " " + split[i+1])
                    else:
                        phrase.append(split[i])

        if self.retList:
            return phrase
        else:#rejoin into one text body in case want to do different tokenisation/one-hot later on
            return " ".join(filter(None, phrase))

    def process_phrases(self):
        """extract bodies"""

        self.bodies = self.data['Body']
        
        #apply preprocessing function to all phrases using list comprehension
        self.preprocessed_phrases = [self.preprocess_phrase(phrase) for phrase in self.data['Body']]
        self.features = [self.extract_features_from_phrase(phrase) for phrase in self.preprocessed_phrases]

        data = {'id': self.data['id'], 'ppBody': self.features, 'label': self.data['Label']}
        self.pp_df = pd.DataFrame(data=data)

In [16]:
#instantiate a new dataset
#run the preprocessing with lowercasing, removing newlines, stemming, and tokenisation using the nltk regex
d = Dataset(dfSpam, preprocessing=["lower","newlines","stemming"], feature_selection=["nltk_tokenize"], retList=False)

In [17]:
#get the dataframe
df = d.pp_df

In [18]:
df.sample(10, random_state=2)['ppBody']

1801.0     subject fwd get it all vlco din x anax vl gra ...
511.0      yes we do purchase uncollected judicial judgem...
12393.0    subject get a costco gold membership this is o...
2514.0     on tue sep 17 2002 at 09 11 01pm 0700 paul pre...
1279.0     subject re your financial security high priori...
2453.0     subject scil 10 call for papers student confer...
22488.0    subject hpl 75 th anniversary celebration hous...
3687.0     subject fwd vlco din valium vlagr xianax som a...
3350.0     subject your domain name flippant house need a...
4747.0     subject no more paying for movies events on ca...
Name: ppBody, dtype: object

In [19]:
#get the spam emails - there are 7328
positive = df[df['label'] == "1"]

In [20]:
#get an equal number of non-spam
negative = df[df['label'] == "0"].sample(7328, random_state=myseed)

In [21]:
"subject fwd get it all".split()

['subject', 'fwd', 'get', 'it', 'all']

In [33]:
def split_doc_and_count(doc):
    return Counter(doc.split())

In [45]:
pos_word_counters = positive['ppBody'].apply(split_doc_and_count).to_frame()
neg_word_counters = negative['ppBody'].apply(split_doc_and_count).to_frame()

In [46]:
pos_word_counters

,ppBody
1.0,"{'save': 3, 'up': 2, 'to': 6, '70': 2, 'on': 3..."
2.0,"{'1': 1, 'fight': 1, 'the': 4, 'risk': 1, 'of'..."
3.0,"{'1': 1, 'fight': 1, 'the': 4, 'risk': 1, 'of'..."
4.0,"{'adult': 5, 'club': 1, 'offers': 2, 'free': 1..."
5.0,"{'i': 2, 'thought': 1, 'you': 3, 'might': 1, '..."
...,...
429.0,"{'subject': 1, 'lucky': 2, 'you': 5, 'congratu..."
430.0,"{'subject': 1, 'new': 8, 'on': 13, 'capitalfm'..."
431.0,"{'subject': 1, 'submit': 4, '600': 4, 'this': ..."
432.0,"{'subject': 1, 'submit': 4, '600': 4, 'this': ..."


In [47]:
pos_word_counters['ppBody']

1.0      {'save': 3, 'up': 2, 'to': 6, '70': 2, 'on': 3...
2.0      {'1': 1, 'fight': 1, 'the': 4, 'risk': 1, 'of'...
3.0      {'1': 1, 'fight': 1, 'the': 4, 'risk': 1, 'of'...
4.0      {'adult': 5, 'club': 1, 'offers': 2, 'free': 1...
5.0      {'i': 2, 'thought': 1, 'you': 3, 'might': 1, '...
                               ...                        
429.0    {'subject': 1, 'lucky': 2, 'you': 5, 'congratu...
430.0    {'subject': 1, 'new': 8, 'on': 13, 'capitalfm'...
431.0    {'subject': 1, 'submit': 4, '600': 4, 'this': ...
432.0    {'subject': 1, 'submit': 4, '600': 4, 'this': ...
433.0    {'subject': 1, 'i': 45, 'can': 17, 't': 8, 'st...
Name: ppBody, Length: 7328, dtype: object

In [48]:
pos_word_counter = pos_word_counters['ppBody'].sum()

In [49]:
pos_word_counter

Counter({'save': 1426,
         'up': 2829,
         'to': 46435,
         '70': 508,
         'on': 10446,
         'life': 1189,
         'insurance': 376,
         'why': 657,
         'spend': 151,
         'more': 4565,
         'than': 1629,
         'you': 31865,
         'have': 8289,
         'quote': 255,
         'savings': 218,
         'ensuring': 33,
         'your': 19707,
         'family': 933,
         's': 8771,
         'financial': 828,
         'security': 1092,
         'is': 17494,
         'very': 1344,
         'important': 457,
         'makes': 198,
         'buying': 134,
         'simple': 671,
         'and': 38993,
         'affordable': 172,
         'we': 9479,
         'provide': 743,
         'free': 4512,
         'access': 708,
         'the': 57407,
         'best': 1850,
         'companies': 1192,
         'lowest': 303,
         'rates': 567,
         'fast': 499,
         'easy': 1063,
         'saves': 16,
         'money': 3929,
         'le

In [50]:
neg_word_counter = neg_word_counters['ppBody'].sum()

# Exploratory Data Analysis

Run some tests on the datasets from `Email-Spam-Dataset`:
- find most common words in spam/not spam emails - plot on histogram
- classify the sentiment of each email as positive/negative and see if there's a pattern in spam/not spam

# Email Classification

## TFIDF

Work in Progress: running TFIDF to classify Email-Spam-Dataset

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english', max_features=10000)
features = tfidf.fit_transform(df.ppBody).toarray()

In [ ]:
features.shape

(18651, 10000)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit

In [ ]:
X = d.pp_df.iloc[:, 0:2]
y = d.pp_df.iloc[:, 2]

ss = ShuffleSplit(n_splits=1, test_size=0.25, random_state=10)
indexes = list(ss.split(X, y))
train_set  = indexes[0][0]
test_set  = indexes[0][1]
Xtrain = X.iloc[train_set, :]
ytrain = y.iloc[train_set]
Xtest = X.iloc[test_set, :]
ytest = y.iloc[test_set]

## Word2Vec

Classify with word2vec (see https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794)

## BERT

Classify with BERT (also using TDS article methods)

## Random Forest

Use RF/ensemble method for some of the classifiers above to see which features work best and which preprocessing methods are good - exploring Phishing-Dataset-for-Machine-Learning features first may be helpful

# Phishing-Dataset-for-Machine-Learning

Inspect this dataset - it is for webpages (https://www.sciencedirect.com/science/article/pii/S0020025519300763?via%3Dihub) but some of the features may be applicable to email classification

Other repos that use this dataset to take inspiration from:
- https://github.com/andpalmier/MLWithPhishing
- https://github.com/rewanthtammana/Detect-phishing-websites-using-ML
- https://www.kaggle.com/code/fadilparves/pishing-detection-using-machine-learning

Random forests used to discover best features - we could apply similar to the Email-Spam-Dataset

## New Feature Extraction

Can we apply the features used in this dataset to Email-Spam-Dataset?

# Comments



# Todo

- [x] Remove NaN id from merged `dfSpam`
- [ ] Tokenise body data
- [ ] Set seed method
- [ ] Stratified Sample?